# Plus TI

## Exploración de datos

### Security Data Science

#### José Daniel Gómez Cabrera 21429


### 2.1 Conteo histórico de fraudes por comercio


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [13]:
df = pd.read_csv("./feature_engineering_work/dataset_feature_engineering.csv", header=0)

### 2.1 Conteo histórico de fraudes por comercio


In [14]:
# Ordenar el dataset por tiempo para mantener la secuencia temporal
df = df.sort_values(by=['unix_time', 'merchant'])

# Crear variable acumulada de fraudes por merchant 
df['fraudes_previos_en_merchant'] = df.groupby('merchant').is_fraud.cumsum() - df.is_fraud

# Para verificar los resultados
print("Primeras filas con los fraudes previos:")
print(df[['merchant', 'is_fraud', 'fraudes_previos_en_merchant']].head(10))

Primeras filas con los fraudes previos:
                             merchant  is_fraud  fraudes_previos_en_merchant
0          fraud_Rippin, Kub and Mann         0                            0
1     fraud_Heller, Gutmann and Zieme         0                            0
2                fraud_Lind-Buckridge         0                            0
3  fraud_Kutch, Hermiston and Farrell         0                            0
4                 fraud_Keeling-Crist         0                            0
5    fraud_Stroman, Hudson and Erdman         0                            0
6               fraud_Rowe-Vandervort         0                            0
7                fraud_Corwin-Collins         0                            0
8                    fraud_Herzog Ltd         0                            0
9   fraud_Schoen, Kuphal and Nitzsche         0                            0


### 2.2 Flag binario: ¿Este comercio ha tenido algún fraude antes?

0 = no tiene antecedentes de fraude

1 = si tiene antecedentes de fraude


In [15]:
df['merchant_con_historial_fraude'] = (df['fraudes_previos_en_merchant'] > 0).astype(int)
# Verificar la creación de la nueva variable
print("Primeras filas con la nueva variable de historial de fraude:")
print(df[['merchant', 'is_fraud', 'fraudes_previos_en_merchant', 'merchant_con_historial_fraude']].head(10))

Primeras filas con la nueva variable de historial de fraude:
                             merchant  is_fraud  fraudes_previos_en_merchant  \
0          fraud_Rippin, Kub and Mann         0                            0   
1     fraud_Heller, Gutmann and Zieme         0                            0   
2                fraud_Lind-Buckridge         0                            0   
3  fraud_Kutch, Hermiston and Farrell         0                            0   
4                 fraud_Keeling-Crist         0                            0   
5    fraud_Stroman, Hudson and Erdman         0                            0   
6               fraud_Rowe-Vandervort         0                            0   
7                fraud_Corwin-Collins         0                            0   
8                    fraud_Herzog Ltd         0                            0   
9   fraud_Schoen, Kuphal and Nitzsche         0                            0   

   merchant_con_historial_fraude  
0                      

### 2.3 Ratio de fraudes por comercio (histórico hasta la transacción)


In [16]:
# Calcular número total de transacciones por comercio hasta la fecha
df['transacciones_previas_en_merchant'] = (
    df.groupby('merchant')
    .cumcount()
)

# Evitar división por cero para no tener errores de ecuaciones
df['ratio_fraudes_en_merchant'] = np.where(
    df['transacciones_previas_en_merchant'] > 0,
    df['fraudes_previos_en_merchant'] / df['transacciones_previas_en_merchant'],
    0
)

### 2.4 Tiempo desde el último fraude en el comercio


In [17]:
# Ordenar el dataframe por tiempo y comercio
df = df.sort_values(['unix_time', 'merchant'])

# Crear marca de tiempo para transacciones fraudulentas
df['timestamp_fraude'] = df['unix_time'].where(df['is_fraud'] == 1)

# Calcular último fraude por comercio
df['ultimo_fraude_en_merchant'] = df.groupby('merchant')['timestamp_fraude'].transform(lambda x: x.ffill().shift())

# Calcular tiempo desde último fraude (en días para mejor interpretación)
df['dias_desde_ultimo_fraude'] = (df['unix_time'] - df['ultimo_fraude_en_merchant']) / (24 * 3600)

# Redondear a días enteros para mejor interpretación
df['dias_desde_ultimo_fraude'] = df['dias_desde_ultimo_fraude'].round(0).astype('Int64')

df['dias_desde_ultimo_fraude'] = df['dias_desde_ultimo_fraude'].fillna(-1) # -1 indica que no hubo fraudes previos en el comercio

# Mostrar solo transacciones fraudulentas para verificar
print("Ejemplo de transacciones fraudulentas:")
print(df[df['is_fraud']==1][['merchant', 'is_fraud', 'timestamp_fraude', 
                             'ultimo_fraude_en_merchant', 'dias_desde_ultimo_fraude']].head())

Ejemplo de transacciones fraudulentas:
                              merchant  is_fraud  timestamp_fraude  \
2449            fraud_Rutherford-Mertz         1      1.325466e+09   
2472  fraud_Jenkins, Hauck and Friesen         1      1.325469e+09   
2523            fraud_Goodwin-Nitzsche         1      1.325474e+09   
2546            fraud_Erdman-Kertzmann         1      1.325475e+09   
2553                fraud_Koepp-Parker         1      1.325477e+09   

      ultimo_fraude_en_merchant  dias_desde_ultimo_fraude  
2449                        NaN                        -1  
2472                        NaN                        -1  
2523                        NaN                        -1  
2546                        NaN                        -1  
2553                        NaN                        -1  


### 2.5 ¿Este usuario ya hizo fraude antes?

Lo más probable es que el número de fraudes históricos esté relacionado a la tarjeta de crédito (cc_num).


In [25]:
# Ordenar el dataframe por tiempo y número de tarjeta
df = df.sort_values(['unix_time', 'cc_num'])

# Calcular fraudes previos por usuario usando transform
df['fraudes_previos_usuario'] = df.groupby('cc_num')['is_fraud'].transform(lambda x: x.shift().cumsum().fillna(0))

# Crear indicador de usuario con historial de fraude
df['usuario_con_historial_fraude'] = (df['fraudes_previos_usuario'] > 0).astype(int)

# Verificar resultados
print(df[['cc_num', 'is_fraud', 'fraudes_previos_usuario', 'usuario_con_historial_fraude']].head(10))

             cc_num  is_fraud  fraudes_previos_usuario  \
0  2703186189652095         0                      0.0   
1      630423337322         0                      0.0   
2    38859492057661         0                      0.0   
3  3534093764340240         0                      0.0   
4   375534208663984         0                      0.0   
5  4767265376804500         0                      0.0   
6    30074693890476         0                      0.0   
7  6011360759745864         0                      0.0   
8  4922710831011201         0                      0.0   
9  2720830304681674         0                      0.0   

   usuario_con_historial_fraude  
0                             0  
1                             0  
2                             0  
3                             0  
4                             0  
5                             0  
6                             0  
7                             0  
8                             0  
9                    

### 2.6 Frecuencia de compras en el comercio por cliente (acumulador)


In [19]:
# columna 'compras_previas_en_merchant_usuario' que cuenta el número de compras previas por usuario en cada comercio
df['compras_previas_en_merchant_usuario'] = (
    df.groupby(['cc_num', 'merchant']).cumcount()
)

### 2.7 ¿Primera vez que el usuario compra en este comercio?


In [20]:
df['primera_vez_en_merchant'] = (df['compras_previas_en_merchant_usuario'] == 0).astype(int)

Con esto podemos realizar la ingeniería de características con nuevas variables


In [21]:
df[['merchant', 'is_fraud', 'fraudes_previos_en_merchant', 'merchant_con_historial_fraude',
    'ratio_fraudes_en_merchant', 'dias_desde_ultimo_fraude',
    'fraudes_previos_usuario', 'usuario_con_historial_fraude',
    'compras_previas_en_merchant_usuario', 'primera_vez_en_merchant']].head(10)

,merchant,is_fraud,fraudes_previos_en_merchant,merchant_con_historial_fraude,ratio_fraudes_en_merchant,dias_desde_ultimo_fraude,fraudes_previos_usuario,usuario_con_historial_fraude,compras_previas_en_merchant_usuario,primera_vez_en_merchant
0,"fraud_Rippin, Kub and Mann",0,0,0,0.0,-1,0.0,0,0,1
1,"fraud_Heller, Gutmann and Zieme",0,0,0,0.0,-1,0.0,0,0,1
2,fraud_Lind-Buckridge,0,0,0,0.0,-1,0.0,0,0,1
3,"fraud_Kutch, Hermiston and Farrell",0,0,0,0.0,-1,0.0,0,0,1
4,fraud_Keeling-Crist,0,0,0,0.0,-1,0.0,0,0,1
5,"fraud_Stroman, Hudson and Erdman",0,0,0,0.0,-1,0.0,0,0,1
6,fraud_Rowe-Vandervort,0,0,0,0.0,-1,0.0,0,0,1
7,fraud_Corwin-Collins,0,0,0,0.0,-1,0.0,0,0,1
8,fraud_Herzog Ltd,0,0,0,0.0,-1,0.0,0,0,1
9,"fraud_Schoen, Kuphal and Nitzsche",0,0,0,0.0,-1,0.0,0,0,1


### Transacciones fraudulentas


In [24]:
df[df['is_fraud'] == 1][['merchant', 'is_fraud', 'fraudes_previos_en_merchant',
                                'merchant_con_historial_fraude', 'ratio_fraudes_en_merchant',
                                'dias_desde_ultimo_fraude', 'fraudes_previos_usuario',
                                'usuario_con_historial_fraude', 'compras_previas_en_merchant_usuario',
                                'primera_vez_en_merchant']].head(25)

,merchant,is_fraud,fraudes_previos_en_merchant,merchant_con_historial_fraude,ratio_fraudes_en_merchant,dias_desde_ultimo_fraude,fraudes_previos_usuario,usuario_con_historial_fraude,compras_previas_en_merchant_usuario,primera_vez_en_merchant
2449,fraud_Rutherford-Mertz,1,0,0,0.000000,-1,0.0,0,0,1
2472,"fraud_Jenkins, Hauck and Friesen",1,0,0,0.000000,-1,0.0,0,0,1
2523,fraud_Goodwin-Nitzsche,1,0,0,0.000000,-1,1.0,1,0,1
2546,fraud_Erdman-Kertzmann,1,0,0,0.000000,-1,1.0,1,0,1
2553,fraud_Koepp-Parker,1,0,0,0.000000,-1,2.0,1,0,1
2937,fraud_Medhurst PLC,1,0,0,0.000000,-1,2.0,1,0,1
3527,fraud_Ruecker Group,1,0,0,0.000000,-1,3.0,1,0,1
3580,fraud_Conroy-Cruickshank,1,0,0,0.000000,-1,3.0,1,0,1
3600,fraud_Koepp-Parker,1,1,1,0.111111,1,4.0,1,1,0
3671,fraud_Strosin-Cruickshank,1,0,0,0.000000,-1,5.0,1,0,1
